<a href="https://colab.research.google.com/github/tiffchu/402-test/blob/main/402_BT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center>pt2. bigrams and trigrams </center>

## Importing the Required Libraries

In [ ]:
#https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#1introduction

#nltk
import nltk
nltk.download("stopwords")
import string
from nltk.corpus import stopwords
import re

import numpy as np
import json
import glob

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords

#vis
!pip install pyLDAvis
#for vis
import pyLDAvis
import pyLDAvis.gensim_models

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


## Preparing the Data

In [10]:
def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = json.load(f)
    return (data)

def write_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)


In [11]:
from nltk.corpus import stopwords
stopwords = stopwords.words("english")
stopwords = [word for word in stopwords if len(word) >= 5]
stopwords.extend(['from', 'go', 'so', 'know', 'subject', 're', 'edu', 'use', 'participant', 'P1', 'P2', 'P3', 'P4', 'P5', 'Facilitator','um','uh'])

In [13]:
print (stopwords)
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

['myself', 'ourselves', "you're", "you've", "you'll", "you'd", 'yours', 'yourself', 'yourselves', 'himself', "she's", 'herself', 'itself', 'their', 'theirs', 'themselves', 'which', "that'll", 'these', 'those', 'being', 'having', 'doing', 'because', 'until', 'while', 'about', 'against', 'between', 'through', 'during', 'before', 'after', 'above', 'below', 'under', 'again', 'further', 'there', 'where', 'other', "don't", 'should', "should've", "aren't", 'couldn', "couldn't", "didn't", 'doesn', "doesn't", "hadn't", "hasn't", 'haven', "haven't", "isn't", 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", "shan't", 'shouldn', "shouldn't", "wasn't", 'weren', "weren't", "won't", 'wouldn', "wouldn't", 'from', 'go', 'so', 'know', 'subject', 're', 'edu', 'use', 'participant', 'P1', 'P2', 'P3', 'P4', 'P5', 'Facilitator', 'um', 'uh']


In [16]:
data = load_data("session1.json")

# Print the first 200 characters of the speech
print(data[:200])

[{'participant': 'Facilitator', 'time': '', 'speech': ''}, {'participant': 'Facilitator', 'time': '', 'speech': ''}, {'participant': 'P3', 'time': '00:00:27)', 'speech': " Okay. So I went out with an idea in mind of a photo that I wanted to take. It was of a road, actually, just the road. Cause that's why I felt like it was, the journey behind me was a road and the journey, another long road. And then as I was coming, this was taken at the park. And as I was coming outta the park off to my right, I saw this and I just like almost slammed on the brakes. Unfortunately, nobody was behind me. I had to get off the side of the road. It just spoke to me. So I just got very emotional because the whole time I've been thinking about what did I want this photo to be, et cetera, et cetera. And it just spoke to me about the fact that it was like I was having to cross from one point to another."}, {'participant': 'P3', 'time': '00:01:15)', 'speech': " I was leaving the safety of the hospital and the

In [18]:
# Function for lemmatization
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return texts_out

# Read data from the file
with open('session1.json', 'r') as file:
    data = json.load(file)

# Extract speech texts from the data
speech_texts = [item.get('speech', '') for item in data]

# Perform lemmatization
lemmatized_texts = lemmatization(speech_texts)

# Print the lemmatized texts
print(lemmatized_texts)

['', '', 'so go idea mind photo want take road actually just road cause feel journey road journey long road then come take park come outta park right see just almost slam brake unfortunately get side road just speak so just get very emotional whole time think want photo et cetera et cetera just speak fact have cross point', 'leave safety hospital then kind of unknown picture represent unknown other side almost bit hill same time log cover moss so transfer log dangerous scary feel leave safety net hospital give fact night event almost pass bathroom still send home next morning very apprehensive transition home fear landing post long haul symptom live fear get case again get sick again end back hospital experience all over again so kinda represent just speak just take photo pleasure give lot time think want go question', 'feel wanna go go wanna', '', 'thank lovely photograph thank explain process incredible just drive along see think speak little bit last week pop', 'picture road actuall

In [22]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

print (data_words[3][0:20])

['leave', 'safety', 'hospital', 'then', 'kind', 'of', 'unknown', 'picture', 'represent', 'unknown', 'other', 'side', 'almost', 'bit', 'hill', 'same', 'time', 'log', 'cover', 'moss']


Bigrams are two words frequently occurring together in the document. Trigrams are 3 words frequently occurring.

In [23]:
#BIGRAMS AND TRIGRAMS
bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=100)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return([bigram[doc] for doc in texts])

def make_trigrams(texts):
    return ([trigram[bigram[doc]] for doc in texts])

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_bigrams)

print (data_bigrams_trigrams[3][0:20])

['leave', 'safety', 'hospital', 'then', 'kind_of', 'unknown', 'picture', 'represent', 'unknown', 'other', 'side', 'almost', 'bit', 'hill', 'same', 'time', 'log', 'cover', 'moss', 'so']


In [24]:
#TF-IDF REMOVAL
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]
# print (corpus[0][0:20])

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.03
words  = []
words_missing_in_tfidf = []
for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow


In [35]:
 id2word = corpora.Dictionary(texts)

 corpus = []
 for text in data_words:
     new = id2word.doc2bow(text)
     corpus.append(new)

 print (corpus[3][0:20])

 word = id2word[[0][:1][0]]
 print (word)

[(1, 2), (9, 1), (10, 1), (11, 2), (12, 1), (16, 2), (17, 1), (21, 1), (26, 1), (28, 2), (29, 1), (30, 1), (31, 1), (32, 1), (33, 2), (35, 1), (36, 1), (38, 3), (39, 1), (40, 1)]
actually


In [36]:
#build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus[:-1],
                                           id2word=id2word,
                                           num_topics=10,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")


In [37]:
#AWWW YEAAA
test_doc = corpus[-1]

vector = lda_model[test_doc]
print (vector)

def Sort(sub_li):
    sub_li.sort(key = lambda x: x[1])
    sub_li.reverse()
    return (sub_li)
new_vector = Sort(vector)
print (new_vector)

[(4, 0.1767607), (5, 0.7011287), (7, 0.04579285), (9, 0.06595753)]
[(5, 0.7011287), (4, 0.1767607), (9, 0.06595753), (7, 0.04579285)]


In [ ]:
#human readable form of the corpus (word, frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:3]]

In [43]:
#lda_model.save("content/test_model.model")


In [42]:
#new_model = gensim.models.ldamodel.LdaModel.load("models/test_model.model")

In [ ]:
test_doc = corpus[-1]

vector = new_model[test_doc]
print (vector)

def Sort(sub_li):
    sub_li.sort(key = lambda x: x[1])
    sub_li.reverse()
    return (sub_li)
new_vector = Sort(vector)
print (new_vector)

[(0, 0.045875695), (1, 0.035836957), (2, 0.43765357), (5, 0.15934144), (7, 0.057556026), (8, 0.072318755), (9, 0.1911866)]
[(2, 0.43765357), (9, 0.1911866), (5, 0.15934144), (8, 0.072318755), (7, 0.057556026), (0, 0.045875695), (1, 0.035836957)]


## Vizualizing the Data

In [45]:
#pyLDAvis.enable_notebook()
#vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
# vis

!pip install "pandas<2.0.0"

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 25.2 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.1.3
    Uninstalling pandas-2.1.3:
      Successfully uninstalled pandas-2.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
pyldavis 3.4.1 requires pandas>=2.0.0, but you have pandas 1.5.3 which is incompatible.


/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.136152 -0.291534       1        1  69.795979
9     -0.081324 -0.247575       2        1  13.572114
4     -0.250514 -0.036842       3        1   5.896171
6     -0.114436  0.263421       4        1   3.566081
2      0.202418 -0.007049       5        1   3.503525
3      0.120401  0.139513       6        1   1.460949
7      0.015814  0.091228       7        1   1.029398
0     -0.047936  0.014636       8        1   0.755397
8      0.006445  0.040810       9        1   0.339541
1      0.012981  0.033391      10        1   0.080846, topic_info=         Term        Freq       Total Category  logprob  loglift
209     laugh   85.000000   85.000000  Default  30.0000  30.0000
169    really  108.000000  108.000000  Default  29.0000  29.0000
107      know  198.000000  198.000000  Default  28.0000  28.0000
97      thank  115.000000  115.000000  Default  27.0000  27.0000
10       feel   77.000000   77.000000  Default  26.0000  26.0000
..        ...         ...         ...      ...      ...      ...
138      stay    0.004096    5.203784  Topic10  -7.1564  -0.0267
55   hospital    0.004096   27.976871  Topic10  -7.1564  -1.7087
134   recover    0.004096    2.822817  Topic10  -7.1564   0.5850
122   fearful    0.004096    1.714171  Topic10  -7.1564   1.0838
62        lot    0.004096   34.816197  Topic10  -7.1564  -1.9274

[470 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
313       1  0.985408           able
0         1  0.949621       actually
0         4  0.037985       actually
802       1  0.583156       adorable
914       3  0.857943          after
...     ...       ...            ...
1060      4  0.487603  weightbearing
1060      6  0.487603  weightbearing
943       5  0.943437        whatnot
496       2  0.827074          while
913       5  0.745287          wound

[384 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 10, 5, 7, 3, 4, 8, 1, 9, 2])

above LDA model has 10 topics.
so each topic is a combination of keywords and each keyword contributes a certain weightage to the topic.

the keywords for each topic and the weightage(importance) of each keyword are shown below:

In [50]:
#basically: a topic (ex. the sixth is about health and eating well)
# times (*) the weight of its significance
lda_model.print_topics()

[(0,
  '0.032*"really" + 0.022*"laugh" + 0.017*"exact" + 0.015*"hearing" + 0.015*"poetic" + 0.013*"nice" + 0.013*"surprising" + 0.010*"seem" + 0.010*"outlook" + 0.010*"inspiring"'),
 (1,
  '0.006*"general" + 0.001*"mean" + 0.001*"okay" + 0.001*"right" + 0.001*"of" + 0.001*"course" + 0.001*"let" + 0.001*"think" + 0.001*"back" + 0.001*"long"'),
 (2,
  '0.025*"health" + 0.023*"feel" + 0.021*"share" + 0.019*"positive" + 0.017*"hear" + 0.017*"manage" + 0.017*"thank" + 0.013*"open" + 0.012*"though" + 0.010*"do"'),
 (3,
  '0.055*"fine" + 0.034*"arm" + 0.021*"no" + 0.020*"completely" + 0.019*"email" + 0.019*"turn" + 0.018*"safe" + 0.015*"road" + 0.015*"forget" + 0.014*"board"'),
 (4,
  '0.037*"really" + 0.026*"next" + 0.026*"show" + 0.019*"feel" + 0.019*"week" + 0.018*"walk" + 0.016*"sign" + 0.014*"family" + 0.014*"one" + 0.013*"light"'),
 (5,
  '0.049*"so" + 0.035*"go" + 0.035*"know" + 0.027*"just" + 0.024*"thank" + 0.020*"think" + 0.019*"laugh" + 0.017*"really" + 0.016*"say" + 0.011*"thing"'

In [60]:
#Model perplexity and topic coherence provide a reliable
# measure to judge how good a given topic model is.

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.496438030327249

Coherence Score:  0.45544672428305144


interpreting my topic model

- Perplexity:  -6.496438030327249
- Coherence Score:  0.4554467242830514

The perplexity and coherence score are two metrics used to evaluate topic models

Perplexity: A measure of how well the model predicts a sample. Lower perplexity values indicate better performance. A perplexity of -6.496 indicates that the model, on average, is making relatively good predictions about unseen data.

Coherence Score: Measures the interpretability of the topics. It quantifies how semantically similar the high-scoring words within a topic are. A coherence score of 0.455 indicates a moderate level of interpretability in the topics generated by the model. Higher coherence scores generally imply more interpretable and coherent topics.

In [74]:
from gensim import corpora, models, similarities, downloader
!pip install smart_open[s3]

from gensim.models.wrappers import LdaMallet

ModuleNotFoundError: ignored

In [75]:
MALLET_PATH = 'C:/mallet-2.0.8/bin/mallet'
lda_mallet = gensim.models.wrappers.LdaMallet(mallet_path=MALLET_PATH, corpus=bow_corpus,
                                              num_topics=TOTAL_TOPICS, id2word=dictionary,
                                              iterations=500, workers=16)


AttributeError: ignored

In [ ]:
!wget -c "https://download.oracle.com/java/17/latest/jdk-17_linux-x64_bin.tar.gz"


In [ ]:
!tar -xvzf jdk-17_linux-x64_bin.tar.gz



In [79]:
import os
os.environ["JAVA_HOME"] = "/content/jdk-17.0.9"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]


In [80]:
!java -version


java version "17.0.9" 2023-10-17 LTS
Java(TM) SE Runtime Environment (build 17.0.9+11-LTS-201)
Java HotSpot(TM) 64-Bit Server VM (build 17.0.9+11-LTS-201, mixed mode, sharing)


In [84]:
MALLET_PATH = 'C:/mallet-2.0.8/bin/mallet'
lda_mallet = gensim.models.wrappers.LdaMallet(mallet_path=MALLET_PATH, corpus=bow_corpus,
                                              num_topics=TOTAL_TOPICS, id2word=dictionary,
                                              iterations=500, workers=16)

AttributeError: ignored